In [1]:
## author: QUAN YU
import requests
import re
import json
from bs4 import BeautifulSoup

# P: pages of a hotel to crawl: [1,352]
P = 20
pages = ["https://www.tripadvisor.com.sg/Hotel_Review-g294264-d1447339-Reviews-or{}-Hard_Rock_Hotel_Singapore-Sentosa_Island.html".format(i) for i in range(0, P * 5, 5)]
for pg in pages[:3]:
    print(pg)

https://www.tripadvisor.com.sg/Hotel_Review-g294264-d1447339-Reviews-or0-Hard_Rock_Hotel_Singapore-Sentosa_Island.html
https://www.tripadvisor.com.sg/Hotel_Review-g294264-d1447339-Reviews-or5-Hard_Rock_Hotel_Singapore-Sentosa_Island.html
https://www.tripadvisor.com.sg/Hotel_Review-g294264-d1447339-Reviews-or10-Hard_Rock_Hotel_Singapore-Sentosa_Island.html


In [3]:
data = []

for url in pages:
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
      
    review = soup.find_all('div', {'class':'review-container'})
    # print(len(review))
    
    # crawling for each review
    for rev in review:
        # info from the main page
        user = rev.find("span", class_="expand_inline scrname")
        user = user.text
    
        member = rev.find("div",{'class':"memberOverlayLink"})
        if(member is None):
            continue
        empty, uid, src = re.split('UID_|-|-SRC_', member['id'])
    
        rate = rev.find("span", class_=re.compile("ui_bubble_rating"))
        rate = rate['class'][1].strip("bubble_")
        
        quote = rev.find("span", class_="noQuotes").text
    
        # info from mouse hover page
        hover = requests.get("https://www.tripadvisor.com.sg/MemberOverlay", params = {"uid": uid})
        overlay = BeautifulSoup(hover.content, "html.parser")
    
        level_ = overlay.find('div', {'class':'badgeinfo'})
        level = level_.span.text if level_ else "none"
    
        cnts = overlay.find_all("li",{"class":'countsReviewEnhancementsItem'})
        out = {
            'contribution': '0', 
            'vote': '0', 
            'city': '0',
            'photo': '0'
          }
        for cnt in cnts:
            item = cnt.text.strip()
            if("Contribution" in item):
                out["contribution"] = item.split()[0]
            elif("visited" in item):
                out["city"] = item.split()[0]
            elif("vote" in item):
                out["vote"] = item.split()[0]
            elif("Photo" in item):
                out["photo"] = item.split()[0]
            
        item = {
            "user": user,
            "uid": uid,
            "rate": rate,
            "quote": quote,
            "level": level,
            "count": out,
            }
        data.append(item)
        
    print(len(data))

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
94
99


In [4]:
print(len(data))
print(data[:3])

99
[{'uid': '53914737C20758B841D5356ED067D5B1', 'count': {'photo': '221', 'city': '173', 'contribution': '710', 'vote': '279'}, 'quote': 'Located on Sentosa', 'user': 'roryjdoyle', 'level': '6', 'rate': '30'}, {'uid': 'DE44459A99806E4FF719F7B8894E0A28', 'count': {'photo': '0', 'city': '71', 'contribution': '52', 'vote': '160'}, 'quote': 'Amazing hotel!', 'user': 'Sordnaxela', 'level': '5', 'rate': '50'}, {'uid': 'F672B733EA52AE5DD4A441BB0AE6D043', 'count': {'photo': '0', 'city': '2', 'contribution': '5', 'vote': '0'}, 'quote': 'A so so experience', 'user': 'jepisco', 'level': '2', 'rate': '30'}]


In [5]:
with open('trip100.json','w') as f:
    json.dump(data,f,indent = 4)